In [148]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [149]:
#All import statements 

import pandas as pd 
import numpy as np 
import matplotlib as plt 
from sklearn.preprocessing import OneHotEncoder

In [150]:
df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [151]:
m, n = df.shape
df = np.array(df)

#Shuffle() all items in a list; it will shuffle all the rows in our array i.e our each individual data entry
np.random.shuffle(df)

#Splitting our data
df_test = df[0:1000].T
y_test = df_test[0]
X_test = df_test[1:n]

df_train = df[1000:m].T
y_train = df_train[0]
X_train = df_train[1:n]

In [152]:
def init_params():
    w1 = np.random.rand(10,784) - 0.5 
    b1 = np.random.rand(10,1) - 0.5
    w2 = np.random.rand(10,10) - 0.5 
    b2 = np.random.rand(10,1) - 0.5
    return w1,b1,w2,b2

def re_lu(z):
    return np.maximum(0,z)

def softmax(z):
    return np.exp(z)/np.sum(np.exp(z))

def one_hot(y):
    one_hot_y = np.zeros((y.size, y.max() + 1))
    one_hot_y[np.arange(y.size), y] = 1
    one_hot_y = one_hot_y.T
    return one_hot_y
    
def fwd_prop(w1,b1,w2,b2,x):
    z1 = w1.dot(x) + b1
    a1 = re_lu(z1)
    z2 = w2.dot(a1) + b2 
    a2 = softmax(z2)
    return z1, a1, z2, a2

def deriv_relu(z):
    return z > 0

def bwd_prop(z1, a1, z2, a2, w2, x, y):
    m  = y.size
    one_hot_y = one_hot(y)
    dz2 = a2 - one_hot_y
    dw2 = (1/m) * dz2.dot(a1.T)
    db2 = (1/m) * np.sum(dz2)
    dz1 = w2.T.dot(dz2) * deriv_relu(z1)
    dw1 = 1/m * dz1.dot(x.T)
    db1 = (1/m) * np.sum(dz1)
    return dw1,db1, dw2, db2 

def update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha):
    w1 = w1 - alpha * dw1
    b1 = b1 - alpha * db1 
    w2 = w2 - alpha * dw2
    b2 = b2 - alpha * db2 
    return w1, b1, w2, b2

In [153]:
def get_predictions(a2):
    return np.argmax(a2,0)

def get_accuracy(predictions, y):
    print(predictions,y)
    return np.sum(predictions == y) / y.size

def gradient_descent(x, y, iterations, alpha):
    w1, b1, w2, b2 = init_params()
    for i in range(iterations):
        z1, a1, z2, a2 = fwd_prop(w1,b1,w2,b2,x)
        dw1, db1, dw2, db2 = bwd_prop(z1, a1, z2, a2, w2, x, y)
        w1, b1, w2, b2 = update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
        if (i % 10) == 0:
            print("Iteration:", i)
            print("Accuracy", get_accuracy(get_predictions(a2), y))
    return w1, b1, w1, b2
        

In [154]:
w1, b1, w2, b2 = gradient_descent(X_train, y_train, 100, 0.1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: overflow encountered in exp
  if sys.path[0] == "":
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == "":


Iteration: 0
[6 0 0 ... 0 0 0] [0 8 2 ... 2 6 4]
Accuracy 0.07590243902439024
Iteration: 10
[0 0 0 ... 0 0 0] [0 8 2 ... 2 6 4]
Accuracy 0.09836585365853659
Iteration: 20
[0 0 0 ... 0 0 0] [0 8 2 ... 2 6 4]
Accuracy 0.09836585365853659
Iteration: 30
[0 0 0 ... 0 0 0] [0 8 2 ... 2 6 4]
Accuracy 0.09836585365853659
Iteration: 40
[0 0 0 ... 0 0 0] [0 8 2 ... 2 6 4]
Accuracy 0.09836585365853659
Iteration: 50
[0 0 0 ... 0 0 0] [0 8 2 ... 2 6 4]
Accuracy 0.09836585365853659
Iteration: 60
[0 0 0 ... 0 0 0] [0 8 2 ... 2 6 4]
Accuracy 0.09836585365853659
Iteration: 70
[0 0 0 ... 0 0 0] [0 8 2 ... 2 6 4]
Accuracy 0.09836585365853659
Iteration: 80
[0 0 0 ... 0 0 0] [0 8 2 ... 2 6 4]
Accuracy 0.09836585365853659
Iteration: 90
[0 0 0 ... 0 0 0] [0 8 2 ... 2 6 4]
Accuracy 0.09836585365853659
